In [47]:
# Imports
from langgraph.graph import StateGraph, END
from typing import TypedDict
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper
from langchain_community.tools import WikipediaQueryRun, ArxivQueryRun
from langchain_classic.chains import RetrievalQA , SimpleSequentialChain
from langchain_classic.memory import ConversationBufferMemory
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.agents import create_agent 
from langchain.tools import tool
from langgraph.checkpoint.memory import MemorySaver
import wikipedia
import arxiv
import os
from dotenv import load_dotenv

In [48]:
# ---------- 1. Setup LLM and memory ----------
llm = ChatOpenAI(temperature=0, model_name="gpt-4")  # or gpt-3.5-turbo
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
embeddings = OpenAIEmbeddings()



In [49]:
# ---------- 2. Load PDF and setup RAG ----------

# Providing Path of file 
pdf_path = r"C:\Users\Admin\Downloads\AWS_Certified AI Practitioner_ Slides.pdf"  

# Loading the pdf 
loader = PyMuPDFLoader(pdf_path)
docs = loader.load()

# Splitting the documents into manageable chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
chunks = splitter.split_documents(docs)

# Creating a vector store from the document chunks and rag chain
vectorstore = FAISS.from_documents(chunks, embedding=embeddings)
rag_qa = RetrievalQA.from_chain_type(llm=llm, retriever=vectorstore.as_retriever())

In [50]:
# ---------- 3. Define tools ----------
@tool
def aws_pdf_qa(query: str) -> str:
    """Answer questions related to AWS AI Practitioner PDF course. Also provide page numbers where applicable.If the question is related to aws(Amazon Web Services) first check is there any explanation available in the PDF, if not then answer from your own knowledge."""
    return rag_qa.run(query)

@tool
def wiki_search(query: str) -> str:
    """Get a brief summary about a topic from Wikipedia"""
    try:
        return wikipedia.summary(query, sentences=2)
    except Exception as e:
        return f"Wikipedia search failed: {e}"

@tool
def arxiv_search(query: str) -> str:
    """Search for research papers on ArXiv and provide clickable links"""
    try:
        search = arxiv.Search(
            query=query,
            max_results=3,
            sort_by=arxiv.SortCriterion.Relevance
        )
        results = [f"{r.title} - [Link]({r.entry_id})" for r in search.results()]
        return "\n".join(results) if results else "No papers found."
    except Exception as e:
        return f"ArXiv search failed: {e}"



In [51]:
# ---------- 4. Initialize agent ----------
tools = [aws_pdf_qa, wiki_search, arxiv_search]

checkpointer = MemorySaver()
agent = create_agent(
    system_prompt="You're an AI assistant that helps with AWS AI Practitioner course content, general knowledge from Wikipedia, and research papers from ArXiv.",
    tools=tools,
    model=llm,
    checkpointer=checkpointer,
)


# Stream to see execution steps
#for chunk in agent.stream({"messages": [{"role": "user", "content": "your question"}]}):
    #print(chunk)

In [53]:
# ---------- 5. Chat loop ----------
config = {"configurable": {"thread_id": "1"}}

while True:
    query = input("\nYou: ")
    if query.lower() in ["exit", "quit"]:
        break
    
    result = agent.invoke({
        "messages": [{"role": "user", "content": query}]
    }, config)
    
    print("\nAgent:", result["messages"][-1].content)

C:\Users\Admin\AppData\Local\Temp\ipykernel_13292\2414167086.py:5: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain-classic 0.1.0 and will be removed in 1.0. Use `invoke` instead.
  return rag_qa.run(query)



Agent: Amazon Q Developer is a tool that integrates with your Integrated Development Environment (IDE) to assist with software development. It can answer questions about AWS development, generate and complete code, scan your code for security vulnerabilities, and help with debugging, optimizations, and improvements. 

It also functions as an AI code companion, similar to GitHub Copilot, supporting many languages such as Java, JavaScript, Python, TypeScript, C#. It provides real-time code suggestions and security scans, and can be used to implement features, generate documentation, and bootstrap new projects. 

Additionally, it can help you understand and manage your cloud infrastructure and AWS costs. However, please note that as of my current knowledge, there might be some confusion with the term "Amazon Q Developer" as it's not a widely recognized or official term in AWS documentation.

Agent: Here's a summary of our conversation about AWS:

1. You asked about "Amazon Q Developer". 